# Дискриминантный анализ

In [506]:
import numpy as np
import math
import itertools
import scipy.stats
import matplotlib.pyplot as plt

## Постановка задачи
Построить и протестировать классификатор с использованием:
- модельных данных;
- данных из репозитория (https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/).

In [ ]:
def get_estimation_mu(D):
    return [sum(D[i])/len(D[i]) for i in range(len(D))]

def get_estimation_sigma(D, estimation_mu, n):
    dim = len(estimation_mu[0])
    deltas = [D[i]-estimation_mu[i] for i in range(len(D))]
    indexes = list(itertools.combinations_with_replacement(range(len(D[0][0])), 2))
    
    estimation_sigma = []
    for j in range(len(n)):
        tmp = np.zeros((dim, dim))
        for i in indexes:
            s = [deltas[j][k][i[0]]*deltas[j][k][i[1]] for k in range(n[j])]
            tmp[i] = sum(s)
        estimation_sigma.append(tmp - np.eye(dim) * tmp + tmp.T)
    return sum(estimation_sigma)/(sum(n)-2)

def get_alpha(mu, sigma):
    return np.dot(np.linalg.inv(sigma), mu[0]-mu[1])

def get_split_value(z, q):
    return z.mean() + math.log(q[1]/q[0])

def get_Mahalanobis_distance_estimate(z, s_z, n, p):
    D = (z[0]-z[1])*(z[0]-z[1])/s_z
    D_H = (sum(n)-p-3)/(sum(n)-2)*D - p*sum(n)/(n[0]*n[1])
    return D, D_H

def get_misclassification_probabilities(D_H, q):
    K = math.log(q[1]/q[0])
    p_2_1 = scipy.stats.norm.cdf((K - D_H/2) / math.sqrt(D_H))
    p_1_2 = scipy.stats.norm.cdf(-(K + D_H/2) / math.sqrt(D_H))
    return p_2_1, p_1_2

In [524]:
def classify(X, a, split_value):
    c = np.dot(X, a)
    mask = c >= split_value
    res = [[],[]]
    for i in range(len(mask)):
        if mask[i]:
            res[0].append(i)
        else:
            res[1].append(i) 
    return res

def do_research(mu, sigma, D, X, y, n):
    q = [n[i]/sum(n) for i in range(len(n))]
    alpha = get_alpha(mu, sigma)
    
    z = np.array([np.dot(D[i], alpha).mean() for i in range(len(n))])
    s_z = np.dot(np.dot(alpha, sigma), alpha)
    d, d_H = get_Mahalanobis_distance_estimate(z, s_z, n, len(D[0][0]))
    
    split_value = get_split_value(z, q)
    
    res = classify(X, alpha, split_value)    
    table_X = np.zeros((2,2))
    table_X[0][1] = sum(y[res[0]] == 1)
    table_X[0][0] = len(res[0]) - table_X[0][1]
    table_X[1][0] = sum(y[res[1]] == 0)
    table_X[1][1] = len(res[1]) - table_X[1][0]
    
    tmp = np.vstack((D[0], D[1]))
    
    res = classify(tmp, alpha, split_value)
    table_D = np.zeros((2,2))
    table_D[0][1] = sum(np.array(res[0]) >= n[0])
    table_D[0][0] = len(res[0]) - table_D[0][1]
    table_D[1][0] = sum(np.array(res[1]) < n[0])
    table_D[1][1] = len(res[1]) - table_D[1][0]
    
    p = get_misclassification_probabilities(d_H, q)
    
    return table_X, table_D, p

### Классификатор с использованием модельных данных

In [537]:
def generate_data(mu, sigma, n, N):
    D = []
    for i in range(len(n)):
        vec = np.random.multivariate_normal(mu[i], sigma, n[i])
        D.append(vec)

    X, y = [], []
    for i in range(N):
        j = np.random.randint(0, 2)
        vec = np.random.multivariate_normal(mu[j], sigma, 1)
        X.append(vec[0])
        y.append(j)

    return D, np.array(X), np.array(y)

In [569]:
n = [250, 250]
N = 1000
mu = np.array([[-1, -1, -1,],
               [1, 1, 1]])
sigma = [[1,0,0],
         [0,1,0],
         [0,0,1]]

np.random.seed(42)
D, X, y = generate_data(mu, sigma, n, N)

table_X, table_D, p = do_research(mu, sigma, D, X, y, n)
print('Table for X:\n', table_X)
print('Table for D:\n', table_D)
print('P =', p)

print()
estimation_mu = get_estimation_mu(D)
estimation_sigma = get_estimation_sigma(D, estimation_mu, n)

table_X, table_D, p = do_research(estimation_mu, estimation_sigma, D, X, y, n)
print('Table for X:\n', table_X)
print('Table for D:\n', table_D)
print('P =', p)

Table for X:
 [[461.  30.]
 [ 20. 489.]]
Table for D:
 [[243.   7.]
 [  7. 243.]]
P = (0.032963389469100976, 0.032963389469100976)

Table for X:
 [[463.  27.]
 [ 18. 492.]]
Table for D:
 [[242.   5.]
 [  8. 245.]]
P = (0.030231350156609996, 0.030231350156609996)


### Классификатор с использованием данных из репозитория

In [607]:
def load_data():
    f = open('data\german.data-numeric', 'r')
    data = []
    for line in f:
        vec = list(map(int, line.split()))
        data.append(vec)
    f.close()
    
    return np.array(data)

def sort_data(data, k):
    indexes_D = list(np.random.choice(len(data), k, replace=False))
    indexes_X = list(set(range(len(data))) - set(indexes_D))
    
    D = [[],[]]
    for elem in data[indexes_D]:
        if elem[-1] == 1:
            D[0].append(elem[0:-1])
        else:
            D[1].append(elem[0:-1])
    n = [len(D[0]), len(D[1])]
    
    X = data[indexes_X, 0:-1]
    y = data[indexes_X, -1]
    return D, X, y, n

In [676]:
k = 100
data = load_data()

D, X, y, n = sort_data(data, k)
print('n =', n)
estimation_mu = get_estimation_mu(D)
estimation_sigma = get_estimation_sigma(D, estimation_mu, n)

table_X, table_D, p = do_research(estimation_mu, estimation_sigma, D, X, y-1, n)
print('Table for X:\n', table_X)
print()
print('Table for D:\n', table_D)
print('P =', p)

n = [65, 35]
Table for X:
 [[518. 139.]
 [117. 126.]]

Table for D:
 [[58.  5.]
 [ 7. 30.]]
P = (0.11379481147802639, 0.3203668545919762)
